In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
author_data = pd.read_csv("../data/raw/author_data.csv")
comment_data = pd.read_csv("../data/raw/comment_data.csv")
gilds_data = pd.read_csv("../data/raw/gildings_data.csv")
threads_data = pd.read_csv("../data/raw/thread_data.csv")

Remove the duplicates - I had to run a modified version of the script since desktop kept shutting down (due to inclement weather; used append flag as a quick fix which resulted in all of the data being written multiple times).

In [ ]:
def drop_duplicates_na(data, key_columns):
    '''Takes in data and key columns as parameters, in this case, key column refers to
    the columns to consider for identifying duplicates. Returns a dataframe where the 
    duplicates (all except for the last occurrence) and columns with NA are removed.'''
    df = data.copy()
    df.drop_duplicates(subset=key_columns, keep='last', inplace=True, ignore_index=True)
    return df.dropna()

In [ ]:
authors_data = drop_duplicates_na(author_data, ['author_ids'])
comments_data = drop_duplicates_na(comment_data, ['comment_ids'])
gilds_data = drop_duplicates_na(gilds_data, ['comment_ids'])
threads_data = drop_duplicates_na(threads_data, ['thread_ids'])

In [ ]:
from pathlib import Path

def save(df, savepath, filename):
    '''Takes in a dataframe and a filename for saving to csv'''
    Path(savepath).mkdir(parents=True, exist_ok=True) #Make the folder if it already doesn't exist
    df.to_csv(f"{savepath}{filename}.csv", header=False)

save(authors_data, "../data/interim/", "author_data")
save(comments_data, "../data/interim/", "comment_data")
save(gilds_data, "../data/interim/", "gildings_data")
save(threads_data, "../data/interim/", "threads_data")

In [ ]:
from datetime import timedelta
import time

def toDays(x, currentime):
    '''Takes in created_utc time (Unix time, in seconds) and current time. Calculates
    age and returns the number of days'''
    d = timedelta(seconds=currentime-x)
    return d.days

def process_dates(data, new_column):
    '''Convert the created_utc time to datetime, fetch the days and place it
    in a new column. Also removes the original created_utc column'''
    df = data.copy()
    now = time.time() #This will differ everytime, but the change will be constant across all the rows.
    df[new_column] = df.created_utc.map(lambda x: toDays(x, now))
    return df.drop(["created_utc"], axis=1)
    
authors = process_dates(authors_data, "acc_age_days")
comments = process_dates(comments_data, "comment_age_days")

For a baseline model, we will work with binary classification (whether a comment is gilded or not). For that, we need to do slightly different transformations, and since there's a large imbalance (gilded vs not gilded comments), we will have to transform the full data and then use stratified sampling.

In [ ]:
comment_gilds = gilds_data.merge(comments, how='outer', on='comment_ids')
comments_all = comment_gilds.merge(authors, how='inner', on='author_ids')
targets = comments_all["gildings"]
comments_all = comments_all.filter(items=["ups", "comment_karma", "link_karma", 
                                        "is_premium", "comment_age_days", "acc_age_days"], axis=1)
targets.fillna(0, inplace=True)
targets = pd.DataFrame(targets.apply(lambda x: 1 if x != 0 else 0))

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

splits = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in splits.split(comments_all, targets):
    pass
X_train = comments_all.iloc[train_index, :]
y_train = targets.iloc[train_index, :]
X_test = comments_all.iloc[test_index, :]
y_test = targets.iloc[test_index, :]

In [ ]:
assert len(X_train) == len(y_train)
assert len(X_test) == len(y_test)

In [ ]:
train_data = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
test_data = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

In [ ]:
save(train_data, "../data/processed/", "train_data_baseline")
save(test_data, "../data/processed/", "test_data_baseline")

In [ ]:
g = sns.PairGrid(comments_all, vars=["gildings", "ups", "acc_age_days", "comment_karma", "link_karma"])
g.map(plt.scatter)